In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, InputLayer, Input, add, dot, maximum, average, multiply, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
import numpy as np
import pandas as pd

In [3]:
tf.test.is_gpu_available()
tf.debugging.set_log_device_placement(True)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [8]:
# reading data
data = pd.read_csv('PrOCTOR_sample_data_all.csv', header=0)
data1 = data.fillna(data.mean()['MolecularWeight':'Salivary Gland'])
data1["target"] = np.where(data1.iloc[:, 1] == "passed", 1, 0)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# x, y variable
X = data1.iloc[:, 2:-1]
y = data1['target']
#print(X)

In [10]:
X = np.array(X)
X = scaler.fit_transform(X)
y = np.array(y)
y_1 = y.reshape(y.shape[0],-1)
y_train = y_1
#y = scaler.fit_transform(X)
des=X[:,:13]
body=X[:,13:]

In [11]:
print(des.shape)
print(body.shape)
print(y.shape)
print(y_1.shape)

(828, 13)
(828, 34)
(828,)
(828, 1)


In [12]:
hid_layer = [100, 100, 50]
print(hid_layer[-1])

50


In [13]:
_, des_col = des.shape
_, body_col = body.shape

In [15]:
class early_Concat:
    def __init__(self, des_shape, body_shape, hid_layer, output_layer):
        self.des_shape = des_shape
        self.body_shape = body_shape
        self.hid_layer = hid_layer  # not used
        self.output_layer = output_layer
        
    def Multi(self):
        # des modality
        des_input = Input(shape = self.des_shape) # input = 17, des_shape = 17
        des_input_x2 = Dense(self.hid_layer[2], activation = 'relu')(des_input)
        des_input_x2_batch = BatchNormalization()(des_input_x2)
        
        # body modality
        body_input = Input(shape = self.body_shape) # input = 30,body_shape = 30
        body_input_x2 = Dense(self.hid_layer[2], activation = 'relu')(body_input)
        body_input_x2_batch = BatchNormalization()(body_input_x2)
        
        ''' DMNN Early Fusion Concatenate model '''
        merge_layer = concatenate([des_input_x2_batch , body_input_x2_batch])
        ''' DMNN Early Fusion Product model '''
        #merge_layer = dot([des_input_x2_batch , body_input_x2_batch])
        ''' DMNN Early Fusion Addtion model '''
        #merge_layer = add([des_input_x2_batch , body_input_x2_batch])
        
        
        x = Dense(hid_layer[0] , activation = 'relu')(merge_layer)
        x = BatchNormalization()(x)

        x = Dense(hid_layer[1] , activation = 'relu')(x)
        x = BatchNormalization()(x)

        x = Dense(hid_layer[2] , activation = 'relu')(x)
        x = BatchNormalization()(x)
        
        x = Dense(self.output_layer , activation = 'sigmoid')(x)
        
        early_fusion = keras.models.Model(inputs = [des_input, body_input], outputs = [x])

        return early_fusion

In [16]:
model = early_Concat(des_col, body_col, hid_layer, 1)
earlyfusion= model.Multi()
print(earlyfusion)

In [17]:
earlyfusion.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 50)           700         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 50)           1750        input_2[0][0]                    
______________________________________________________________________________________________

In [14]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# 모델설정
sm = SMOTE(random_state=202004)
loo = LeaveOneOut()
kfold = KFold(n_splits = 5, shuffle = True, random_state = 111)
kfold.get_n_splits(X)

5

In [15]:
def predict_change(predict):
    y_predict = []
    for i in range(len(predict)):
        y_predict = np.append(y_predict, predict[i])
    
    y_predict = np.array(y_predict)
    y_predict = y_predict.reshape(y_predict.shape[0],-1)
    return y_predict

In [16]:
dict_y = dict()

class train_model:
    def __init__(self, x, y, model, pd_y, pd_sm_y, pd_cw_y, pd_smcw_y):
        self.X = x
        self.y = y
        self.model = model
        self.pd_y = pd_y
        self.pd_sm_y = pd_sm_y
        self.pd_cw_y = pd_cw_y
        self.pd_smcw_y = pd_smcw_y
        
    def train_based(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_y
    
    def train_smote(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_sm_y
    
    def train_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_cw_y
    
    def train_sm_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
    
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)
    
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_smcw_y

In [17]:
model = early_Concat(des_col, body_col, hid_layer, 1)
earlyfusion= model.Multi()
print(earlyfusion)

early_y, early_sm_y, early_cw_y, early_smcw_y = [], [], [], []
output_early = train_model(X, y, earlyfusion, dict_y, dict_y, dict_y, dict_y)

# sum model training
#early_y = output_early.train_based()
#early_sm_y = output_early.train_smote()
early_cw_y = output_early.train_weight()
#early_smcw_y = output_early.train_sm_weight()

TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
  ...
    to  
  ['...']
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/re

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Train on 662 samples
Epoch 1/50
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/d

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_distributed_function_20888 in device /job:localhost/replica:0/task:0/device:GPU:0
663/663 [==============================] - 3s 4ms/sample - loss: 0.2005 - accuracy: 0.9638
Epoch 2/50
663/663 [==============================] - 0s 117us/sample - loss: 0.1074 - accuracy: 0.9593
Epoch 3/50
663/663 [==============================] - 0s 120us/sample - loss: 0.0530 - accuracy: 0.9759
Epoch 4/50
663/663 [==============================] - 0s 122us/sample - loss: 0.0309 - accuracy: 0.9879
Epoch 5/50
663/663 [==============================] - 0s 119us/sample - loss: 0.0365 - accuracy: 0.9879
Epoch 6/50
663/663 [==============================] - 0s 122us/sample - loss: 0.0317 - accuracy: 0.9910
Epoch 7/50
663/663 [======================

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

In [18]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math

class Evaluation:
    def __init__(self, pred, y):
        self.y_pred = pred
        self.y = y
        
    def matrix(self):
        y_1 = self.y.reshape(self.y.shape[0],-1)
        y_pred = np.array(self.y_pred)
        y_pred = y_pred.reshape(y_pred.shape[0], -1)
        
        y_classify = []
        for i in range(len(self.y_pred)):
            if self.y_pred[i] >= 0.5:
                a = 1.
                y_classify.append(a)
            else:
                a = 0.
                y_classify.append(a)
        
        fpr,tpr,threshold = roc_curve(y_1 , y_pred, pos_label = 1)
        precision, recall, threshold = precision_recall_curve(y_1, y_pred, pos_label = 1)
        
        roc_auc = auc(fpr,tpr)
        auprc = auc(recall, precision)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        F1 = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall)
        # binary 
        accuracy = accuracy_score(y_1, y_classify)
        mcc = matthews_corrcoef(y_1, y_classify) 
        g_mean = geometric_mean_score(y_1, y_classify)
        confusion = confusion_matrix(y_1, y_classify)
        
        tn, fp, fn, tp = confusion.ravel()
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = tp / (tp + fp)
        fnr = fn / (fn + tp)
        fpr = fp / (fp + tn)

        confu_precision = ppv 
        confu_recall = tpr # sensitivity
        confu_f1 = 2 * ((ppv * tpr) / (ppv + tpr))
        confu_accuracy = (tp + tn) / (tp + tn + fp + fn)
        confu_mcc = ((tp * tn)-(fp-fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        confu_g_mean = math.sqrt(tpr * tnr)
        Optimized_precision = (confu_accuracy - abs(tnr-tpr)) / (tnr + tpr)

        print('공통 \nAUC :',roc_auc) # pb
        print("AUPRC :", auprc) # pb
        print("Optimized precision :", Optimized_precision)

        print("\nfunction 사용\nAccuracy :", accuracy)
        print("Precision(pb) :", mean_precision) #pb
        print("Recall(pb) :", mean_recall) # pb
        print("F1 score(pb) :", F1) #pb
        print("MCC :", mcc)
        print("G-mean :", g_mean)
        
        print("\nConfusion_matrix 사용 \n", confusion)
        print("Accuracy :", confu_accuracy)
        print("Precision :", confu_precision) 
        print("Recall :", confu_recall) 
        print("F1 score :", confu_f1) 
        print("MCC :", confu_mcc)
        print("G-mean :", confu_g_mean)
        
        return roc_auc, auprc, Optimized_precision, accuracy, mean_precision, mean_recall, F1, mcc, g_mean, confu_accuracy, confu_precision, confu_recall, confu_f1, confu_mcc, confu_g_mean;

In [20]:
#early_y_result = Evaluation(predict_change(early_y), y_1)
#early_sm_y_result = Evaluation(early_sm_y, y)
early_cw_y_result = Evaluation(predict_change(early_cw_y), y)
#early_smcw_y_result = Evaluation(early_smcw_y, y)

print("#early model 결과 \n")
print("\n-----based model-----\n")
#print(early_y_result.matrix())

print("\n-----smote model-----\n")
#print(early_sm_y_result.matrix())

print("\n-----class weight model-----\n")
print(early_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
#print(early_smcw_y_result.matrix())

#early model 결과 


-----based model-----


-----smote model-----


-----class weight model-----

공통 
AUC : 0.9529276052616891
AUPRC : 0.9949366210677619
Optimized precision : 0.41553697699681225

function 사용
Accuracy : 0.9396135265700483
Precision(pb) : 0.9905346768593017
Recall(pb) : 0.5888698766767442
F1 score(pb) : 0.7386277717134153
MCC : 0.6396452640755487
G-mean : 0.8305679707504404

Confusion_matrix 사용 
 [[ 51  20]
 [ 30 727]]
Accuracy : 0.9396135265700483
Precision : 0.9732262382864793
Recall : 0.9603698811096433
F1 score : 0.9667553191489362
MCC : 0.6503419664108856
G-mean : 0.8305679707504403
(0.9529276052616891, 0.9949366210677619, 0.41553697699681225, 0.9396135265700483, 0.9905346768593017, 0.5888698766767442, 0.7386277717134153, 0.6396452640755487, 0.8305679707504404, 0.9396135265700483, 0.9732262382864793, 0.9603698811096433, 0.9667553191489362, 0.6503419664108856, 0.8305679707504403)

-----smote + class weight model-----



C:\Users\user\anaconda3\envs\test\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [33]:
early_y_set = []

for i in range(20):
    model = early_Concat(des_col, body_col, hid_layer, 1)
    earlyfusion= model.Multi()
    print(earlyfusion)

    early_y, early_sm_y, early_cw_y, early_smcw_y = [], [], [], []
    output_early = train_model(X, y, earlyfusion, dict_y, dict_y, dict_y, dict_y)

    # concatenate model training
    #early_y = output_early.train_based()
    #early_sm_y = output_early.train_smote()
    #early_cw_y = output_early.train_weight()
    early_smcw_y = output_early.train_sm_weight()
    
    #early_y_result = Evaluation(predict_change(early_y), y_1)
    #early_sm_y_result = Evaluation(early_sm_y, y)
    #early_cw_y_result = Evaluation(predict_change(early_cw_y), y)
    early_smcw_y_result = Evaluation(predict_change(early_smcw_y), y)
    
    early_y_set = np.append(early_y_set, early_smcw_y_result.matrix())
    print(len(early_y_set))
    
early_y_np = np.array(early_y_set)

early_y_np = early_y_np.reshape(-1,15)
early_y_np.shape

TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
  ...
    to  
  ['...']
Train on 1200 samples
Epoch 1/50
1200/1200 [==============================] - 1s 1ms/sample - loss: 1.3289 - accuracy: 0.7233
Epoch 2/50
1200/1200 [==============================] - 0s 127us/sample - loss: 0.5678 - accuracy: 0.8142
Epoch 3/50
1200/1200 [==============================] - 0s 125

1214/1214 [==============================] - 0s 122us/sample - loss: 0.0395 - accuracy: 0.9811
Epoch 32/50
1214/1214 [==============================] - 0s 122us/sample - loss: 0.0502 - accuracy: 0.9819
Epoch 33/50
1214/1214 [==============================] - 0s 122us/sample - loss: 0.1075 - accuracy: 0.9703
Epoch 34/50
1214/1214 [==============================] - 0s 119us/sample - loss: 0.1025 - accuracy: 0.9580
Epoch 35/50
1214/1214 [==============================] - 0s 119us/sample - loss: 0.0597 - accuracy: 0.9753
Epoch 36/50
1214/1214 [==============================] - 0s 119us/sample - loss: 0.0540 - accuracy: 0.9819
Epoch 37/50
1214/1214 [==============================] - 0s 122us/sample - loss: 0.0546 - accuracy: 0.9761
Epoch 38/50
1214/1214 [==============================] - 0s 120us/sample - loss: 0.0648 - accuracy: 0.9728
Epoch 39/50
1214/1214 [==============================] - 0s 120us/sample - loss: 0.0577 - accuracy: 0.9802
Epoch 40/50
1214/1214 [==========================

C:\Users\user\anaconda3\envs\test\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
  ...
    to  
  ['...']
Train on 1200 samples
Epoch 1/50
1200/1200 [==============================] - 1s 987us/sample - loss: 1.4091 - accuracy: 0.7100
Epoch 2/50
1200/1200 [==============================] - 0s 119us/sample - loss: 0.5870 - accuracy: 0.8342
Epoch 3/50
1200/1200 [==============================] - 0s 1

Epoch 12/50
1218/1218 [==============================] - 0s 124us/sample - loss: 0.1202 - accuracy: 0.9466
Epoch 13/50
1218/1218 [==============================] - 0s 125us/sample - loss: 0.1006 - accuracy: 0.9573
Epoch 14/50
1218/1218 [==============================] - 0s 124us/sample - loss: 0.1232 - accuracy: 0.9442
Epoch 15/50
1218/1218 [==============================] - 0s 125us/sample - loss: 0.1170 - accuracy: 0.9433
Epoch 16/50
1218/1218 [==============================] - 0s 123us/sample - loss: 0.1300 - accuracy: 0.9376
Epoch 17/50
1218/1218 [==============================] - 0s 120us/sample - loss: 0.1019 - accuracy: 0.9507
Epoch 18/50
1218/1218 [==============================] - 0s 122us/sample - loss: 0.1578 - accuracy: 0.9368
Epoch 19/50
1218/1218 [==============================] - 0s 125us/sample - loss: 0.1131 - accuracy: 0.9491
Epoch 20/50
1218/1218 [==============================] - 0s 124us/sample - loss: 0.1331 - accuracy: 0.9401
Epoch 21/50
1218/1218 [==============

...

C:\Users\user\anaconda3\envs\test\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
  ...
    to  
  ['...']
Train on 1200 samples
Epoch 1/50
1200/1200 [==============================] - 1s 994us/sample - loss: 1.2635 - accuracy: 0.7458
Epoch 2/50
1200/1200 [==============================] - 0s 123us/sample - loss: 0.5170 - accuracy: 0.8075
Epoch 3/50
1200/1200 [==============================] - 0s 1

1214/1214 [==============================] - 0s 122us/sample - loss: 0.0423 - accuracy: 0.9852
Epoch 32/50
1214/1214 [==============================] - 0s 120us/sample - loss: 0.0606 - accuracy: 0.9794
Epoch 33/50
1214/1214 [==============================] - 0s 120us/sample - loss: 0.0308 - accuracy: 0.9885
Epoch 34/50
1214/1214 [==============================] - 0s 123us/sample - loss: 0.0820 - accuracy: 0.9745
Epoch 35/50
1214/1214 [==============================] - 0s 120us/sample - loss: 0.0856 - accuracy: 0.9662
Epoch 36/50
1214/1214 [==============================] - 0s 120us/sample - loss: 0.0622 - accuracy: 0.9703
Epoch 37/50
1214/1214 [==============================] - 0s 123us/sample - loss: 0.0497 - accuracy: 0.9811
Epoch 38/50
1214/1214 [==============================] - 0s 122us/sample - loss: 0.0851 - accuracy: 0.9753
Epoch 39/50
1214/1214 [==============================] - 0s 120us/sample - loss: 0.0636 - accuracy: 0.9728
Epoch 40/50
1214/1214 [==========================

C:\Users\user\anaconda3\envs\test\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


(20, 15)

In [22]:
def result(data):
    auc = data[:,0]
    auprc = data[:,1]
    op = data[:,2]
    acc = data[:,3]
    precision = data[:,4]
    recall = data[:,5]
    f1 = data[:,6]
    mcc = data[:,7]
    g_mean = data[:,8]
    confu_acc = data[:,9]
    confu_precision = data[:,10]
    confu_recall = data[:,11]
    confu_f1 = data[:,12]
    confu_mcc = data[:,13]
    confu_g_mean = data[:,14]

    print('공통 \nAUC :\n',"[평균 :",format(auc.mean(), ".5f"), "] [표준편차 :",format(auc.std(), ".5f"),"]") # pb
    print("AUPRC :\n", "[평균 :",format(auprc.mean(), ".5f"), "] [표준편차 :",format(auprc.std(), ".5f"),"]")
    print("Optimized precision :\n","[평균 :",format(op.mean(), ".5f"), "] [표준편차 :",format(op.std(), ".5f"),"]")

    print("\nfunction 사용\nAccuracy :\n","[평균 :",format(acc.mean(), ".5f"), "] [표준편차 :",format(acc.std(), ".5f"),"]")
    print("Precision(pb) :\n", "[평균 :",format(precision.mean(), ".5f"), "] [표준편차 :",format(precision.std(), ".5f"),"]")
    print("Recall(pb) :\n", "[평균 :",format(recall.mean(), ".5f"), "] [표준편차 :",format(recall.std(), ".5f"),"]")
    print("F1 score(pb) :\n","[평균 :",format(f1.mean(), ".5f"), "] [표준편차 :",format(f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(mcc.mean(), ".5f"), "] [표준편차 :",format(mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(g_mean.mean(), ".5f"), "] [표준편차 :",format(g_mean.std(), ".5f"),"]")
    
    print("\nConfusion matrix 사용 \nAccuracy :\n", "[평균 :",format(confu_acc.mean(), ".5f"), "] [표준편차 :",format(confu_acc.std(), ".5f"),"]")
    print("Precision :\n", "[평균 :",format(confu_precision.mean(), ".5f"), "] [표준편차 :",format(confu_precision.std(), ".5f"),"]")
    print("Recall :\n", "[평균 :",format(confu_recall.mean(), ".5f"), "] [표준편차 :",format(confu_recall.std(), ".5f"),"]")
    print("F1 score :\n", "[평균 :",format(confu_f1.mean(), ".5f"), "] [표준편차 :",format(confu_f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(confu_mcc.mean(), ".5f"), "] [표준편차 :",format(confu_mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(confu_g_mean.mean(), ".5f"), "] [표준편차 :",format(confu_g_mean.std(), ".5f"),"]")
    

In [34]:
result(early_y_np)

공통 
AUC :
 [평균 : 0.88694 ] [표준편차 : 0.01641 ]
AUPRC :
 [평균 : 0.98350 ] [표준편차 : 0.00341 ]
Optimized precision :
 [평균 : 0.43211 ] [표준편차 : 0.02118 ]

function 사용
Accuracy :
 [평균 : 0.88007 ] [표준편차 : 0.01139 ]
Precision(pb) :
 [평균 : 0.98065 ] [표준편차 : 0.00284 ]
Recall(pb) :
 [평균 : 0.57164 ] [표준편차 : 0.00976 ]
F1 score(pb) :
 [평균 : 0.72221 ] [표준편차 : 0.00801 ]
MCC :
 [평균 : 0.46902 ] [표준편차 : 0.03727 ]
G-mean :
 [평균 : 0.79773 ] [표준편차 : 0.02782 ]

Confusion matrix 사용 
Accuracy :
 [평균 : 0.88007 ] [표준편차 : 0.01139 ]
Precision :
 [평균 : 0.97069 ] [표준편차 : 0.00486 ]
Recall :
 [평균 : 0.89590 ] [표준편차 : 0.01214 ]
F1 score :
 [평균 : 0.93175 ] [표준편차 : 0.00687 ]
MCC :
 [평균 : 0.49301 ] [표준편차 : 0.03273 ]
G-mean :
 [평균 : 0.79773 ] [표준편차 : 0.02782 ]
